In [1]:
import pickle
import gensim
import numpy as np
import math

In [2]:
#reading the variables from file : dictionary and tdf/corpus
with open(r"C:\Users\kanis\Major-Project\Tokens\tkn01.pkl","rb") as fp:
    data = pickle.load(fp)

dictionary = data[0]
termdocfreq = data[1]
finallemma = data[2]
feedtodict = data[3]

In [3]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=termdocfreq,
                                       id2word=dictionary,
                                       num_topics=num_topics,random_state=100, chunksize=100, passes=10, per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[termdocfreq]

[(0,
  '0.048*"be" + 0.034*"learn" + 0.029*"word" + 0.023*"language" + 0.015*"more" '
  '+ 0.015*"thing" + 0.015*"phrase" + 0.014*"memorize" + 0.014*"t" + '
  '0.013*"help"'),
 (1,
  '0.038*"be" + 0.030*"language" + 0.028*"phrase" + 0.026*"learn" + '
  '0.023*"read" + 0.021*"try" + 0.020*"thing" + 0.018*"t" + 0.017*"important" '
  '+ 0.016*"Mastering"'),
 (2,
  '0.044*"be" + 0.039*"learn" + 0.032*"language" + 0.028*"word" + 0.014*"t" + '
  '0.013*"memorize" + 0.013*"help" + 0.012*"more" + 0.011*"time" + '
  '0.010*"study"'),
 (3,
  '0.046*"be" + 0.045*"learn" + 0.027*"language" + 0.019*"t" + 0.019*"phrase" '
  '+ 0.017*"thing" + 0.017*"word" + 0.015*"try" + 0.015*"most" + 0.015*"read"'),
 (4,
  '0.045*"be" + 0.041*"learn" + 0.035*"language" + 0.029*"word" + 0.016*"t" + '
  '0.012*"help" + 0.012*"new" + 0.011*"memorize" + 0.011*"re" + 0.011*"s"'),
 (5,
  '0.057*"be" + 0.043*"try" + 0.039*"easy" + 0.039*"thing" + 0.038*"watch" + '
  '0.037*"show" + 0.037*"movie" + 0.034*"language" + 0.02

In [4]:
from nltk import tokenize

file10 = open(r"C:\Users\kanis\Major-Project\Intermediate\itm01.txt","r+")

Lines = file10.readlines()
#print(Lines)
x = ""
for line in Lines:
    x = x + line
#print(x)
tksen = tokenize.sent_tokenize(x)
print(len(tksen))

35


In [5]:
#lexical sentence weighting 

x=lda_model.show_topics(num_topics=num_topics, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

dict = {}


for topic,words in topics_words:
    for word in words :
        dict[word] = 0
#print(dict)

s_w = {}
for sentence in tksen:
    count = 0
    for word in sentence.split() :
        
        if word in dict:
            count = count+1
    s_w[sentence] = count/len(sentence)
keys = list(s_w.keys())
values = list(s_w.values())
sorted_value_index =np.argsort(values)
sorted_dict = {keys[i]: values[i] for i in sorted_value_index}



#print(s_w)
avgvalue = 0
for ele in s_w:
    avgvalue += s_w[ele]
    
avgvalue = (avgvalue)/len(s_w)
summarylex = []
for ele in s_w:
    if s_w[ele] > avgvalue :
        summarylex.append(ele)
#print(len(summarylex))

0::['be', 'learn', 'word', 'language', 'more']
1::['be', 'language', 'phrase', 'learn', 'read']
2::['be', 'learn', 'language', 'word', 't']
3::['be', 'learn', 'language', 't', 'phrase']
4::['be', 'learn', 'language', 'word', 't']
5::['be', 'try', 'easy', 'thing', 'watch']
6::['be', 'learn', 'word', 'language', 't']
7::['learn', 'be', 'language', 'word', 'phrase']
8::['be', 'learn', 'language', 'word', 't']
9::['be', 'learn', 'word', 'language', 'memorize']



In [6]:
#sliding window sentence weighting 
window_size = 9
sw_sw = {}
for sentence in tksen :
    weight = -1e9
    words  =  sentence.split()
    i = 0 
    j = 9
    if j>len(words):
        count = 0
        for word in words :
            if word in dict :
                count += 1
        weight = max(weight,count/window_size)
        break
    while j<len(words):
        count = 0
        for k in range(j-i):
            if words[k] in dict :
                count += 1
        weight = max(weight,count/window_size)
        i = i+1
        j = j+1
    sw_sw[sentence] = weight
#print(sw_sw)
avgvalue = 0
for ele in sw_sw:
    avgvalue += sw_sw[ele]
    
avgvalue = (avgvalue)/len(sw_sw)
slidingwindowsumm = []
for ele in sw_sw:
    if sw_sw[ele] > avgvalue :
        slidingwindowsumm.append(ele)
print(slidingwindowsumm)

['Possibly the easiest thing you can do is watch television shows or movies in the language you are trying to learn.', "To make things easier, try to watch shows or movies whose plots you are already familiar withâ€”like kids' cartoons or dubbed versions of English movieâ€”knowing the context will help you to decipher the meanings of words and phrases.", 'As your language level progresses, you can move on to more advanced reading material like newspapers and magazines.', 'When you read out loud, rather than silently, you can work on both your overall reading comprehension and pronunciation skills.', 'Itâ€™s more important to master basic phrases youâ€™ll use frequently than it is to start off by learning the alphabet or the perfect sentence structure.', "Therefore, it's important that when you learn a word, you learn the pronunciation simultaneously.", 'This is pretty much backwardsâ€”if you want to learn a language quickly, you should learn how to converse first.']


In [7]:
#coherence and score 
from gensim.models import CoherenceModel
# instantiate topic coherence model
import numpy as np
newcorpus = []
for lis in finallemma :
    for w in lis :
        if len(w)>2:
            newcorpus.append(w)

cm = CoherenceModel(model=lda_model, texts=feedtodict, dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = cm.get_coherence()
    print('Coherence: ', coherence_lda)                
# getting topic coherence score 


#getting the perplexity
print('\nPerplexity: ', lda_model.log_perplexity(termdocfreq))

Coherence:  0.42157048179208756

Perplexity:  -5.0261946621355165


In [8]:
#import pickle 
#import pyLDAvis
#import os 
#import pyLDAvis.gensim


#num_topics = 10
import pyLDAvis
import pyLDAvis.gensim 

# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary,sort_topics = 'false')
#vis
#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

#LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary)
    

In [9]:
import pandas as pd

In [10]:
#Finding the dominant topics 
import random
def format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts=tksen):
    # Init output
    
    dp = []
    pc = []
    tk = []
    
    
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        #print(row_list)
        row = row_list[0] if ldamodel.per_word_topics else row_list 
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                x = random.randint(0, 9)
                topic_num=(topic_num+x)%10
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                #print(topic_keywords + " "+str(topic_num)+" "+str(prop_topic))
                #sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])], ignore_index=True)
                #df = pd.DataFrame({'Dominant_Topic': topic_num,'Perc_Contribution':prop_topic,'Topic_Keywords':topic_keywords})
                #sent_topics_df = pd.concat([sent_topics_df, df], ignore_index=True)
                dp.append(topic_num)
                pc.append(prop_topic)
                tk.append(topic_keywords)
            else:
                break
   
    sent_topics_df = pd.DataFrame({'Dominant_Topic' : dp, 'Perc_contribution':pc, 'topic_keywords':tk })
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts = tksen)
df_len=len(df_topic_sents_keywords.index)
df_topic_sents_keywords=df_topic_sents_keywords.drop(df_topic_sents_keywords.index[len(tksen):])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_len=len(df_topic_sents_keywords.index)
print(df_len)
df_dominant_topic.head(df_len)

35


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,7,0.549967,"learn, be, language, word, phrase, t, thing, h...",Possibly the easiest thing you can do is watch...
1,1,3,0.699977,"be, learn, language, t, phrase, thing, word, t...","Try to avoid subtitles, so you donâ€™t develop..."
2,2,5,0.774979,"be, try, easy, thing, watch, show, movie, lang...","To make things easier, try to watch shows or m..."
3,3,4,0.819981,"be, learn, language, word, t, help, new, memor...","As your language level progresses, you can mov..."
4,4,7,0.849981,"learn, be, language, word, phrase, t, thing, h...","When you read out loud, rather than silently, ..."
5,5,4,0.871413,"be, learn, language, word, t, help, new, memor...","Rather than trying to translate each phrase, f..."
6,6,6,0.887487,"be, learn, word, language, t, help, important,...",Mastering everyday conversation will let you h...
7,7,0,0.899990,"be, learn, word, language, more, thing, phrase...",Itâ€™s more important to master basic phrases ...
8,8,0,0.909991,"be, learn, word, language, more, thing, phrase...",Mastering basic vocabulary is probably one of ...
9,9,9,0.918173,"be, learn, word, language, memorize, t, help, ...","Even if you can't understand whole sentences, ..."


In [11]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(len(tksen))


,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.966663,"be, learn, word, language, more, thing, phrase...",This is especially important when you are havi...
1,1,0.957137,"be, language, phrase, learn, read, try, thing,...","If you are up for a challenge, memorize all th..."
2,2,0.974997,"be, learn, language, word, t, memorize, help, ...",Learning a language within the context of cult...
3,3,0.974283,"be, learn, language, t, phrase, thing, word, t...",If you can visit and spend some time in a coun...
4,4,0.962495,"be, learn, language, word, t, help, new, memor...","Too often, people spend all of their time stud..."
5,5,0.969996,"be, try, easy, thing, watch, show, movie, lang...",Weâ€™ll breakdown each app below:\nPeople ofte...
6,6,0.967853,"be, learn, word, language, t, help, important,...","In addition, looking up the word and using it ..."
7,7,0.965380,"learn, be, language, word, phrase, t, thing, h...",Carrying a dictionary will allow you to find t...
8,8,0.970964,"be, learn, language, word, t, s, n, thing, imp...","But when they say five years, they probably me..."
9,9,0.972724,"be, learn, word, language, memorize, t, help, ...",You might find yourself in situations where yo...


In [12]:
sent_topics_sorteddf_mallet.to_csv(r"C:\Users\kanis\Major-Project\PreClusteringData\mrdfet.csv")
#saving the most representative document for each topic 

In [13]:
df_dominant_topic.to_csv(r"C:\Users\kanis\Major-Project\PreClusteringData\dt.csv")
#saving the topic wise information for each sentence or document

In [14]:
import csv 


#making the list of clusters with their sentences classfied accordingly 
listofclusters = [None] * num_topics
for i in range(num_topics):
    listofclusters[i] = []

with open(r'C:\Users\kanis\Major-Project\PreClusteringData\dt.csv', 'r',encoding="utf8") as csvfile :
    datareader = csv.reader(csvfile)
    for row in datareader:
        if row[0] == '':
            continue
        listofclusters[int(row[2])].append(row[5]) # saving the sentence(row5) in its dominant topic(row2) cluster
     
    
#loop for reprsentation
i = 0
for item in listofclusters :
    #print("topic cluster no. "+str(i))
    #for sentence in item:
        #print(sentence)
    #print()
    i = i + 1
    
#print(listofclusters)

In [15]:
import re 
import nltk 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [16]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [17]:
listofsentence_vectors = []
sentence_vectors = []
for sentences in listofclusters:
    for i in sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    listofsentence_vectors.append(sentence_vectors)


In [18]:
import sklearn

In [19]:
# similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
k = 0

listofsimmatrix = []
for sentences in listofclusters:
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #We will use Cosine Similarity to compute the similarity between a pair of sentences.


#And initialize the matrix with cosine similarity scores.
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(listofsentence_vectors[k][i].reshape(1,100), listofsentence_vectors[k][j].reshape(1,100))[0,0]
    k=k+1
    listofsimmatrix.append(sim_mat)
#print(len(listofsimmatrix))

In [20]:
listofsubsum = [None] * num_topics
for i in range(num_topics):
    listofsubsum[i] = []

k = 0
for sim_mat in listofsimmatrix:
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(listofclusters[k])), reverse=True)
    for i in range(int(len(ranked_sentences)/3)):
        listofsubsum[k].append(ranked_sentences[i][1])
    k=k+1
#print(listofsubsum)
#print()
#print(listofclusters)


#generating the final list
finallist = []
for lis in listofsubsum :
    for sent in lis :
        if len(sent)>0:
            finallist.append(sent)
#print(finallist)

In [21]:
#creating dictionary to arrange summaryin chronological order
dictofsent = {}
k = 0
for sent in tksen :
    dictofsent[sent] = k
    k=k+1
summarytosave=sorted(finallist, key=dictofsent.get)
#print(summarytosave)

sample = ""
for sent in summarytosave:
    sample += sent 
    sample += ' '

listsave = []
listsave.append(sample)




In [22]:
#lexrank 
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [23]:
clustersummary = [None] * num_topics
for i in range(num_topics):
    clustersummary[i] = []
   
k =0
for cluster in listofclusters:
    sentences = ""
    for sent in cluster :
        sentences += sent
        sentences += ' '
    p = PlaintextParser.from_string(sentences, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summ = summarizer(p.document, int(len(cluster)/3))
    for s in summ:
        clustersummary[k].append(s)
    k=k+1

In [24]:
finallexsumm = ""
for subsum in clustersummary:
    if len(subsum)==0:
        continue
    for sent in subsum:
        if len(str(sent))>0:
            finallexsumm += str(sent)
            finallexsumm += ' '
#print(finallexsumm)


listsave.append(finallexsumm)
listsave.append(summarylex)
listsave.append(slidingwindowsumm)
with open(r"C:\Users\kanis\Major-Project\Generated Summary\gen_sum01.pkl","wb") as fp:
    pickle.dump(listsave,fp)
with open(r"C:\Users\kanis\Major-Project\Generated Summary\gen_sum01.pkl","rb") as fp:
    data = pickle.load(fp)
    print(data[0])
    print()
    print(data[1])

When you read out loud, rather than silently, you can work on both your overall reading comprehension and pronunciation skills. Rather than trying to translate each phrase, force yourself to think in the language you're reading. Mastering basic vocabulary is probably one of the most important things you can do when learning a new language. Therefore, it's important that when you learn a word, you learn the pronunciation simultaneously. Start off with clear goals. Each day learn a new set of ten numbers, keep going each day until you are satisfied with how high you can count. In turn, better reading and pronunciation can help you memorize words with greater ease. Speaking with a real, live person will help you to feel much more motivated about learning the language than staring at a book or computer screen. Thatâ€™s all a normal part of the language-learning process. 

Itâ€™s more important to master basic phrases youâ€™ll use frequently than it is to start off by learning the alphabet 